In [3]:
import torch
import torch.nn as nn
import numpy as np

# Master branch

In [4]:
class MasterNet(nn.Module):
    def __init__(self):
        super(MasterNet, self).__init__()
        
    def forward(self,x):
        return x

# Acoustic Branch

Inputs for acoustic branch will be N x 40 where N [1,33]  
Time step: (2, 10) (seconds?)  
N: relative duration after feature extraction

In [5]:
class AcousticNet(nn.Module):
    def __init__(self, num_conv_layers = 3, kernel_size = 2, conv_width = 32):
        super(AcousticNet, self).__init__()
        self.batch_conv = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=conv_width, kernel_size=kernel_size), 
            nn.MaxPool1d(kernel_size = 2),
            nn.ReLU()
        )
        self.gru = nn.GRU(input_size=19,hidden_size=32,num_layers=2)
        self.mean_pool = nn.AvgPool1d(kernel_size=2)
        
    def forward(self, x):
        x = self.conv(x)
        print('conv', x.shape)
        x = self.maxpool(x)
        print('max pool', x.shape)
        x = self.relu(x)
        print('relu', x.shape)
        x, _ = self.gru(x)
        print('gru', x.shape)
        x = self.mean_pool(x)
        print('mean pool', x.shape)
        return x

In [12]:
# Test dummy input
net = AcousticNet()
test_vec = torch.randn(10, 1, 40)
output = net(test_vec)
print(f'Shape of output: {output.shape}')
# assert output.shape[-1] == 16

conv torch.Size([10, 32, 39])
max pool torch.Size([10, 32, 19])
relu torch.Size([10, 32, 19])
gru torch.Size([10, 32, 32])
mean pool torch.Size([10, 32, 16])
Shape of output: torch.Size([10, 32, 16])


# Lexical Branch

In [5]:
# implement GRU (or transformer)
class LexicalNet(nn.Module):
    def __init__(self, num_gru_layers = 2):
        super(LexicalNet, self).__init__()
        # implement GRU (or transformer)
        self.gru = nn.GRU(input_size=300,hidden_size=32,num_layers=num_gru_layers)
        self.mean_pool = nn.AvgPool1d(kernel_size=2) 
        
    def forward(self, x):
        x, _ = self.gru(x)
        x = self.mean_pool(x)
        return x

In [6]:
# Test dummy input
net = LexicalNet(num_gru_layers = 3)
test_vec = torch.randn(10, 1, 300)
output = net(test_vec)
assert output.shape[-1] == 16